In [ ]:
import geopandas as gpd #A more flexible package to work with geospatial data in python 
from itertools import product
import matplotlib.pyplot as plt
import networkx as nx # networkx package 
import numpy as np
from osgeo import ogr #GDAL package 
import osmnx as ox 
import pandas as pd #Base package for data analysis and manipulation 
from pyproj import CRS #package for your projection management 
import random
import rasterio as rio
from rasterio import mask
import rasterstats as rs
import shapely.geometry #python package for basic spatial operation 


In [ ]:
#set the city names
g_namen = ["Utrecht"]
#g_namen = ["Den_Haag"]

#download city geometries from file
project_gemeenten = gpd.read_file(r"data\boundaries\UrbanRunner_Areas.geojson") ## dit komt van de cbs gemeentegrenzen. Beschrijven bij data

#set additional geometry column of extent
project_gemeenten["extent"] = project_gemeenten.envelope.buffer(2000).envelope.to_crs("EPSG:4326")

#set index to gemeentenaam column
project_gemeenten.set_index('gemeentenaam', inplace=True)

#project to WGS84
project_gemeenten.to_crs("EPSG:4326", inplace=True)


In [ ]:
# get network
for g in g_namen:
    globals()[f"{g}_nodes"] = gpd.read_file(f'data/graphs/with_costs/{g}_graph_costs.gpkg', driver = "GPKG", layer= 'nodes').set_index('osmid') 
    globals()[f"{g}_edges"] = gpd.read_file(f'data/graphs/with_costs/{g}_graph_costs.gpkg', driver = "GPKG", layer= 'edges').set_index(['u', 'v', 'key'])
    edges = globals()[f"{g}_edges"]
    
    #changes here were made in later stage, hence why it is inlcuded in this part
    edges['cost_UHI'] = edges['UHI_mean'] / 3
    edges['cost_greenspace'][(edges['landuse'].isnull()) & (edges['leisure'].isnull()) & (edges['natural'].isnull())] = 1
    edges['cost_disruptions'] = 0
    edges['cost_disruptions'][(edges['footway'] == 'crossing') |(edges['cycleway'] == 'crossing') |(edges['barrier_obstnodes_u'] == 'gate')|(edges['barrier_obstnodes_u'] == 'wicket_gate')|(edges['barrier_obstnodes_u'] == 'stile')|(edges['barrier_obstnodes_u'] == 'turnstile')|(edges['barrier_obstnodes_u'] == 'full-height_turnstile')] = 0.5
    edges['cost_disruptions'][(edges['highway_obstnodes_u'] == 'traffic_signals')] = 1
    
    #final cost column are added before analysis to gain more flexibility in changing the cost function
    edges['final_cost_willemijn'] = ((0.7*edges['cost_UHI']) + (0.9*edges['cost_greenspace']) + (0.5*edges['cost_grade']) + (0.1*edges['cost_disruptions']) + (0.3*edges['cost_surface_prefpaved'])) * edges['length']
    edges['final_cost_rachid'] = ((0.5*edges['cost_waterpoints']) + (0.7*edges['cost_UHI']) + (0.9*edges['cost_greenspace']) + (0.3*edges['cost_disruptions']) + (0.1*edges['cost_surface_prefunpaved'])) * edges['length']
    globals()[f"{g}_edges"] = edges
    globals()[f"{g}_network_both"] = ox.graph_from_gdfs(globals()[f"{g}_nodes"],globals()[f"{g}_edges"])

In [ ]:
def get_random_XY_in_polygon(poly): ## manier om XY coordinaten te pakken binnen een polygon
    minx, miny, maxx, maxy = poly.bounds
    while True:
        p = shapely.geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if poly.contains(p):
            return p.coords[0][0], p.coords[0][1]

In [ ]:
#getting random nodes
n_routes = 1000
for g in g_namen:
    X_list = []
    Y_list = []    
    for i in range((n_routes*2)):
        x, y = get_random_XY_in_polygon(project_gemeenten.to_crs('epsg:28992').loc[g].geometry)
        X_list.append(x)
        Y_list.append(y)
    random_nodes = ox.nearest_nodes(globals()[f"{g}_network_both"], X_list, Y_list)
    source_nodes = random_nodes[0:n_routes]
    goal_nodes = random_nodes[n_routes:(n_routes*2)]

In [ ]:
SameNodesCount = 0
for routenr in range(n_routes):
    #print(routenr)
    source = source_nodes[routenr]
    goal = goal_nodes[routenr]
    if source == goal:
        print('yeah, what is the chance to randomly select the same node?')
        SameNodesCount += 1
        continue
    # calculate routes ## !! Error handling needs to be included in case no route can be found!
    WillemijnRoute = nx.dijkstra_path(G=Utrecht_network_both, source=source, target=goal, weight='final_cost_willemijn')
    RachidRoute = nx.dijkstra_path(G=Utrecht_network_both, source=source, target=goal, weight='final_cost_rachid')
    shortestroute = nx.dijkstra_path(G=Utrecht_network_both, source=source, target=goal, weight='length')
    
    # extract edge attributes of the routes
    #CRstats = pd.DataFrame(ox.utils_graph.get_route_edge_attributes(Gs, coolestroute)) 
    #SRstats = pd.DataFrame(ox.utils_graph.get_route_edge_attributes(Gs, shortestroute)) 
    WRstats = pd.DataFrame(ox.utils_graph.get_route_edge_attributes(Utrecht_network_both, WillemijnRoute))
    RRstats = pd.DataFrame(ox.utils_graph.get_route_edge_attributes(Utrecht_network_both, RachidRoute))
    SRstats = pd.DataFrame(ox.utils_graph.get_route_edge_attributes(Utrecht_network_both, shortestroute))
    if routenr == 0:
        RouteStats = pd.DataFrame([[source, goal, 
                            sum(WRstats.UHI_mean*WRstats.length)/sum(WRstats.length),                             
                            sum(RRstats.UHI_mean*RRstats.length)/sum(RRstats.length),                                   
                            sum(SRstats.UHI_mean*SRstats.length)/sum(SRstats.length),

                            sum(WRstats.length - (WRstats.cost_greenspace*WRstats.length))/sum(WRstats.length),
                            sum(RRstats.length - (RRstats.cost_greenspace*RRstats.length))/sum(RRstats.length),
                            sum(SRstats.length - (SRstats.cost_greenspace*SRstats.length))/sum(SRstats.length),

                            sum(WRstats.grade*WRstats.length)/sum(WRstats.length),
                            sum(SRstats.grade*SRstats.length)/sum(SRstats.length),

                            sum(WRstats.cost_surface_prefpaved*WRstats.length)/sum(WRstats.length),
                            sum(RRstats.cost_surface_prefunpaved*RRstats.length)/sum(RRstats.length),
                            sum(SRstats.cost_surface_prefpaved*SRstats.length)/sum(SRstats.length),
                            sum(SRstats.cost_surface_prefunpaved*SRstats.length)/sum(SRstats.length),

                            len(RRstats[RRstats.cost_waterpoints == 0]),
                            len(SRstats[SRstats.cost_waterpoints == 0]),
                            len(RRstats[RRstats.index_waterpoints1].unique()),
                            len(SRstats[SRstats.index_waterpoints1].unique()),

                            len(WRstats[WRstats.cost_disruptions > 0]),
                            len(RRstats[RRstats.cost_disruptions > 0]),
                            len(SRstats[SRstats.cost_disruptions > 0]),

                            len(WRstats[WRstats.cost_disruptions == 1])/2,
                            len(RRstats[RRstats.cost_disruptions == 1])/2,
                            len(SRstats[SRstats.cost_disruptions == 1])/2,

                            
                            sum(WRstats.length), sum(RRstats.length), sum(SRstats.length)]],
                        columns=['source','target',
                            'WR_UHI','RR_UHI','SR_UHI',
                            'WR_greenspace', 'RR_greenspace', 'SR_greenspace', 
                            'WR_grade', 'SR_grade',
                            'WR_prefpaved','RR_prefunpaved','SR_prefpaved','SR_prefunpaved', 
                            'RR_water','SR_water', 'RR_waterpoints', 'SR_waterpoints'
                            'WR_disruption','RR_disruption','SR_disruption',
                            'WR_trafficsignals','RR_trafficsignals','SR_trafficsignals', 
                            'WR_length', 'RR_length', 'SR_length'])
    else:
        RouteStats = RouteStats.append(pd.DataFrame([[source, goal, 
                            sum(WRstats.UHI_mean*WRstats.length)/sum(WRstats.length),                             
                            sum(RRstats.UHI_mean*RRstats.length)/sum(RRstats.length),                                   
                            sum(SRstats.UHI_mean*SRstats.length)/sum(SRstats.length),

                            sum(WRstats.length - (WRstats.cost_greenspace*WRstats.length))/sum(WRstats.length),
                            sum(RRstats.length - (RRstats.cost_greenspace*RRstats.length))/sum(RRstats.length),
                            sum(SRstats.length - (SRstats.cost_greenspace*SRstats.length))/sum(SRstats.length),

                            sum(WRstats.grade*WRstats.length)/sum(WRstats.length),
                            sum(SRstats.grade*SRstats.length)/sum(SRstats.length),

                            sum(WRstats.cost_surface_prefpaved*WRstats.length)/sum(WRstats.length),
                            sum(RRstats.cost_surface_prefunpaved*RRstats.length)/sum(RRstats.length),
                            sum(SRstats.cost_surface_prefpaved*SRstats.length)/sum(SRstats.length),
                            sum(SRstats.cost_surface_prefunpaved*SRstats.length)/sum(SRstats.length),

                            len(RRstats[RRstats.cost_waterpoints == 0]),
                            len(SRstats[SRstats.cost_waterpoints == 0]),
                            len(RRstats[RRstats.index_waterpoints1].unique()),
                            len(SRstats[SRstats.index_waterpoints1].unique()),

                            len(WRstats[WRstats.cost_disruptions > 0]),
                            len(RRstats[RRstats.cost_disruptions > 0]),
                            len(SRstats[SRstats.cost_disruptions > 0]),

                            len(WRstats[WRstats.cost_disruptions == 1])/2,
                            len(RRstats[RRstats.cost_disruptions == 1])/2,
                            len(SRstats[SRstats.cost_disruptions == 1])/2,

                            
                            sum(WRstats.length), sum(RRstats.length), sum(SRstats.length)]],
                        columns=['source','target',
                            'WR_UHI','RR_UHI','SR_UHI',
                            'WR_greenspace', 'RR_greenspace', 'SR_greenspace', 
                            'WR_grade', 'SR_grade',
                            'WR_prefpaved','RR_prefunpaved','SR_prefpaved','SR_prefunpaved', 
                            'RR_water','SR_water', 'RR_waterpoints', 'SR_waterpoints'
                            'WR_disruption','RR_disruption','SR_disruption',
                            'WR_trafficsignals','RR_trafficsignals','SR_trafficsignals', 
                            'WR_length', 'RR_length', 'SR_length']),
                  ignore_index=True)

In [ ]:
RouteStats.plot.scatter('RR_prefunpaved', 'SR_prefunpaved')

In [ ]:
RouteStats

In [ ]:
RouteStats.to_excel('data/routestats.xlsx')